Author: Luca Pappalardo
</br>Geospatial Analytics, Master degree in Data Science and Business Informatics, University of Pisa

# Geospatial Analytics - Lesson 3: Spatial and Mobility data

In this lesson, we will learn how to handle and explore spatial data in Python using folium and scikit-mobility.

1. [Exploring datasets](#exploring)
2. [GPS traces](#gpstraces)
3. [Geotagged posts](#geotaggedposts)
4. [Skmob data module](#datamodule)
5. [Road Networks and OSMnx](#roadnetworks)
    - [Get administrative boundaries](#getboundaries)
    - [Download road networks](#downloadnetworks)
    - [Analyze road networks](#analyzenetworks)
6. [Points of Interest and Geospatial entities](#pois)

<a id="exploring"></a>
# Exploring datasets in scikit-mobility

One of the use cases for [scikit-mobility](https://github.com/scikit-mobility/scikit-mobility) is the exploratory data analysis of mobility datasets, which includes the visualization of trajectories and flows. 

To this end, both `TrajDataFrame` and `FlowDataFrame` have methods that allow the user to produce interactive visualizations generated using [folium](https://python-visualization.github.io/folium/). 

## Visualizing trajectories

A `TrajDataFrame` has three main plotting methods: 
- `plot_trajectory` plots a line connecting the trajectory points on a map;
- `plot_stops` plots the location of stops on a map; 
- `plot_diary` plots the sequence of visited locations over time.

### Plot trajectories

Method `plot_trajectory` plots the time-ordered trajectory points connected by straight lines on a map. 

If the column `uid`is present and contains more than one object, the trajectory points are first grouped by `uid` and then sorted by `datetime`.

Large `TrajDataFrame`s with many points can be computationally intensive to visualize. Two arguments can be used to reduce the amount of data to plot:
- `max_users` limits the number of objects whose trajectories should be plotted
- `max_points` limits the number of trajectory points per object to plot, i.e., if necessary an object's trajectory will be down-sampled and at most `max_points` points will be plotted.

The plot style can be customized via arguments to specify the color, weight, and opacity of the trajectory lines, as well as the type of map tiles to use. 
The user can also plot markers denoting the start points and the end points of the trajectory. 

The `plot_trajectory` method, as well as all the other plotting methods, return a `folium.Map` object, which can be used by other `folium` and scikit-mobility functions to visualize additional data on the same map. A `folium.Map` object can be passed to a plotting method via the argument `map_f`. If `map_f` is `None`, the trajectories are plotted on a new map.


In [ ]:
# import the library
import skmob
import pandas as pd

In [ ]:
skmob.__version__

In [ ]:
# From a list
data_list = [[1, 39.984044, 116.319236, '2008-10-23 13:53:05'],
             [1, 39.984198, 116.319322, '2008-10-23 13:53:06'],
             [1, 39.974224, 116.319402, '2008-10-23 13:53:11'],
             [1, 39.974211, 116.219389, '2008-10-23 13:53:16']]

In [ ]:
tdf = skmob.TrajDataFrame(data_list, user_id=0, latitude=1, longitude=2, datetime=3)
tdf

In [ ]:
map_f = tdf.plot_trajectory(zoom=12, start_end_markers=False, hex_color='red')
map_f

<a id='gpstraces'></a>
## GPS traces

In [ ]:
# create a TrajDataFrame from a dataset of GPS traces
url = "https://github.com/scikit-mobility/tutorials/raw/master/mda_masterbd2020/data/geolife_sample.txt.gz"
tdf = skmob.TrajDataFrame.from_file(url)
print(type(tdf))
tdf.head()

In [ ]:
tdf.uid.unique()

In [ ]:
map_f = tdf.plot_trajectory(zoom=12, max_users=2) #hex_color='#000000')
map_f

NOTE: if trajectories represent abstract mobility, such as movements extracted from social media posts or mobile phone records, straight lines may appear that do not consider walls, buildings, and similar structures on the road network. 

## Geotagged traces

In [ ]:
url = 'https://snap.stanford.edu/data/loc-brightkite_totalCheckins.txt.gz'
df = pd.read_csv(url, sep = '\t', header = 0, nrows = 100000, 
                 names = ['user', 'check-in_time', 'latitude', 'longitude', 'location id'])
tdf_ci = skmob.TrajDataFrame(df, latitude = 'latitude', longitude = 'longitude', 
                          datetime = 'check-in_time', user_id = 'user').sort_values(by = 'datetime')

In [ ]:
tdf_ci.head()

In [ ]:
tdf_ci.plot_trajectory(max_users=2, start_end_markers=False, zoom=3)

In [ ]:
ni_tdf_ci = tdf_ci[(tdf_ci['lat'] == 0.0) & (tdf_ci['lng'] == 0.0)]
ni_tdf_ci

## Visualizing flows

A `FlowDataFrame` has two main plotting methods:
- `plot_tessellation` plots the tessellation's tiles on a geographic map;
- `plot_flows` plots, on a geographic map, the lines connecting the centroids of the tessellation's tiles between which flows are present.

### Plot tessellation

Method `plot_tessellation` plots the `GeoDataFrame` associated with a `FlowDataFrame` on a geographic map. 

Large tessellations with many tiles can be computationally intensive to visualize. The argument `maxitems` can be used to limit the number of tiles to plot (default: -1, which means that all tiles are displayed).

The plot style can be customized via arguments to specify the color, opacity of the tiles, and the type of map tiles to use. The argument `popup_features` allows to enhance the plot's interactivity displaying popup windows that appear when the user clicks on a tile and includes information contained in the columns of the tessellation's `GeoDataFrame` specified in the argument's list.

In [ ]:
import geopandas as gpd
from skmob import FlowDataFrame

In [ ]:
url = "https://raw.githubusercontent.com/scikit-mobility/tutorials/master/mda_masterbd2020/data/NY_counties_2011.geojson"
tessellation = gpd.read_file(
    url) # load a tessellation
tessellation.head()

In [ ]:
from skmob.utils.plot import plot_gdf

In [ ]:
plot_gdf(tessellation)

In [ ]:
url = "https://github.com/scikit-mobility/tutorials/raw/master/mda_masterbd2020/data/NY_commuting_flows_2011.csv"
# create a FlowDataFrame from a file and a tessellation
fdf = skmob.FlowDataFrame.from_file(url, tessellation=tessellation, tile_id='tile_id')
fdf.head()

In [ ]:
fdf.plot_tessellation(popup_features=['tile_id','population'], style_func_args={'fillColor':'black', 'color':'red'})

### Plot flows

Method `plot_flows` plots the flows on a geographic map as lines between the centroids of the tiles in the `FlowDataFrame`'s tessellation. 

Large `FlowDataFrame`s with many origin-destination pairs can be computationally intensive to visualize. The argument `min_flow` can be used to specify that only flows larger than `min_flow` should be displayed. 

The thickness of each line is a function of the flow and can be specified via the arguments `flow_weight`, `flow_exp` and `style_function`. 

The plot style can be further customized via arguments to specify the color, opacity of the flow lines, and the type of map tiles to use. 

The arguments `flow_popup` and `tile_popup` allow enhancing the plot's interactivity displaying popup windows that appear when the user clicks on a flow line or a circle in an origin location, respectively, and include information on the flow or the flows from a location.

In [ ]:
fdf.plot_flows(min_flow=100)

The user can also visualize the tessellation and the flows in the same plot.

In [ ]:
map_f = fdf.plot_tessellation(style_func_args={'fillColor':'blue', 'color':'blue'}) 
fdf.plot_flows(map_f=map_f, min_flow=50)

<a id="datamodule"></a>
## The scikit-mobility data module

The scikit-mobility data module allows you to retrieve mobility datasets via an easy-to-use interface. Look at the documentation [here](https://scikit-mobility.github.io/scikit-mobility/reference/data.html).

The data module API consists of two main functions: `list_datasets` and `load_dataset`. 
- `list_datasets` shows the datasets already available in the repository;
- `load_dataset `retrieves the requested dataset and directly outputs it into a skmob-friendly data structure.

___

The scikit-mobility data types and format to describe mobility data are the following:

- trajectory $\rightarrow$ `TrajDataFrame`
- flow $\rightarrow$ `FlowDataFrame`
- shape $\rightarrow$ `GeoDataFrame`
- auxiliar $\rightarrow$ `DataFrame`

In [ ]:
!pip install pooch

In [ ]:
from skmob.data.load import load_dataset, list_datasets

### `list_datasets()`

`list_datasets` shows the names and information of the datasets already available in the scikit-mobility repository;

In [ ]:
# list all the datasets' names without details

print(list_datasets())

In [ ]:
# list all the datasets with details into a Pandas DataFrame

dataset_details = list_datasets(details=True)
pd.DataFrame.from_dict(dataset_details, orient='index')

In [ ]:
# list all the datasets of type "flow" with details into a Pandas DataFrame

dataset_details = list_datasets(data_types='flow', details=True)
pd.DataFrame.from_dict(dataset_details, orient='index')

In [ ]:
#list all the datasets of type "trajectory" or "shape" with details into a Pandas DataFrame

dataset_details = list_datasets(data_types=['trajectory', 'shape'], details=True)
pd.DataFrame.from_dict(dataset_details, orient='index')

### `load_dataset()`

`load_dataset` retrieves the requested dataset and directly outputs it into a skmob-friendly data structure.

#### Load the Foursquare NYC trajectory dataset

In [ ]:
# loading the trajectory dataset into a TrajDataFrame

tdf_nyc = load_dataset("foursquare_nyc", drop_columns=True)
print(type(tdf_nyc))

In [ ]:
print(tdf_nyc._info)

In [ ]:
tdf_nyc[:3]

#### Load the Shape of New York City's boundaries

In [ ]:
# loading the shape into a GeoDataFrame

shape_nyc = load_dataset("nyc_boundaries")
print(type(shape_nyc))

In [ ]:
print(shape_nyc._info)

In [ ]:
shape_nyc = shape_nyc.to_crs("epsg:4326")
shape_nyc[:5]

#### Let's use the loaded datasets!

In [ ]:
# plot the trajectories of the first 3 users over the shapefile of New York City

from skmob.utils.plot import plot_gdf

map_f = tdf_nyc.plot_trajectory(max_users=3)
map_f = plot_gdf(shape_nyc, zoom=10, map_f=map_f)

map_f

<a id="roadnetworks"></a>
## Road Networks

[***OSMnx***](https://www.sciencedirect.com/science/article/pii/S0198971516303970) is a Python package to retrieve, model, analyze, and visualize street networks from [OpenStreetMap](https://www.openstreetmap.org/). Users can download and model walkable, drivable, or bikeable urban networks with a single line of Python code, and then easily analyze and visualize them. You can just as easily download and work with amenities/points of interest, building footprints, elevation data, street bearings/orientations, and network routing. 

In [ ]:
import osmnx as ox

<a id="getboundaries"></a>
### Get administrative place boundaries and shapefiles
With OSMnx, you can download place shapes from OpenStreetMap (as geopandas `GeoDataFrame`s) in one line of Python code – and project them to UTM (zone calculated automatically) and visualize in just one more line of code:

In [ ]:
city = ox.geocode_to_gdf('Rome, Italy')
ax = ox.project_gdf(city).plot()
_ = ax.axis('off')

You can just as easily get other place types, such as neighborhoods, boroughs, counties, states, or nations – any place geometry in OpenStreetMap:

In [ ]:
place1 = ox.geocode_to_gdf('Pisa, Italy')
place2 = ox.geocode_to_gdf('Italy')
place3 = ox.geocode_to_gdf('Manhattan, New York City, New York, USA')
place4 = ox.geocode_to_gdf('Tuscany, Italy')

In [ ]:
place4.plot()

Or you can pass multiple places into a single query to save a single shapefile or geopackage from their geometries. You can do this with cities, states, countries or any other geographic entities:

In [ ]:
places = ox.geocode_to_gdf(['Italy', 'Austria', 'Germany'])
places = ox.project_gdf(places)
ax = places.plot()
_ = ax.axis('off')

<a id="downloadnetworks"></a>
### Download and model street networks

OSMnx lets you download street network data and build topologically-corrected street networks, project and plot the networks. The street networks are directed and preserve one-way directionality. You can download a street network by providing OSMnx any of the following:

- a bounding box
- a lat-long point plus a distance
- an address plus a distance
- a polygon of the desired street network’s boundaries
- a place name or list of place names

You can also specify several different network types:

- `drive` – get drivable public streets (but not service roads)
- `drive_service` – get drivable public streets, including service roads
- `walk` – get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
- `bike` – get all streets and paths that cyclists can use
- `all` – download all (non-private) OSM streets and paths
- `all_private` – download all OSM streets and paths, including private-access ones

#### From bounding box

In [ ]:
G = ox.graph_from_bbox(43.7400, 43.6918, 10.3601, 10.4477, network_type='walk') #'drive')
G_projected = ox.project_graph(G)
ox.plot_graph(G_projected)

To access geometries easily, we can convert the graph into a `GeoDataFrame`

In [ ]:
# you can convert your graph to node and edge GeoPandas GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [ ]:
gdf_nodes.head()

In [ ]:
gdf_nodes.plot()

In [ ]:
gdf_edges.head()

In [ ]:
gdf_edges.plot()

You can create a graph from node/edge `GeoDataFrame`s, as long as `gdf_nodes` is indexed by osmid and `gdf_edges` is multi-indexed by $u$, $v$, $key$ (following normal `MultiDiGraph` structure). This allows you to load graph node/edge shapefiles or GeoPackage layers as `GeoDataFrame`s then convert to a `MultiDiGraph` for graph analytics.

In [ ]:
# convert node/edge GeoPandas GeoDataFrames to a NetworkX MultiDiGraph
G2 = ox.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=G.graph)
G2

#### From lat-long points

In [ ]:
G = ox.graph_from_point((43.7400, 10.3601), dist=1750, network_type='all')
ox.plot_graph(G)

#### From address:

In [ ]:
G = ox.graph_from_address('Largo Bruno Pontecorvo 1, Pisa, Italy', network_type='drive')
ox.plot_graph(G)

#### From Polygon

In [ ]:
place1

In [ ]:
G = ox.graph_from_polygon(place1['geometry'].iloc[0], network_type='drive')
ox.plot_graph(G)

#### From place name

In [ ]:
G = ox.graph_from_place('Milan, Italy', network_type='drive')
ox.plot_graph(G)

You can just as easily request a street network within a borough, county, state, or other geographic entity. You can also pass a list of places (such as several neighboring cities) to create a unified street network within them. This list of places can include strings and/or structured key:value place queries:

In [ ]:
places = ['Los Altos, California, USA',
          {'city':'Los Altos Hills', 'state':'California'},
          'Loyola, California']
G = ox.graph_from_place(places, network_type='drive')
ox.plot_graph(G)

<a id="analyzenetworks"></a>
### Analyze street networks

In [ ]:
# what sized area does our network cover in square meters?
G = ox.graph_from_place("Piedmont, California, USA", network_type="drive")
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
graph_area_m

In [ ]:
# show some basic stats about the network
ox.basic_stats(G_proj, area=graph_area_m, clean_int_tol=15)

#### Visualize street centrality
Here we plot the street network and color its edges (streets) by their relative closeness centrality.

In [ ]:
import networkx as nx

In [ ]:
# convert graph to line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(G))
nx.set_edge_attributes(G, edge_centrality, "edge_centrality")

In [ ]:
# color edges in original graph with closeness centralities from line graph
ec = ox.plot.get_edge_colors_by_attr(G, "edge_centrality", cmap="inferno")
fig, ax = ox.plot_graph(G, edge_color=ec, edge_linewidth=2, node_size=0)

#### Routing

In [ ]:
# impute missing edge speeds and calculate edge travel times with the speed module
G = ox.speed.add_edge_speeds(G)
G = ox.speed.add_edge_travel_times(G)

In [ ]:
# get the nearest network nodes to two lat/lng points with the distance module
orig = ox.distance.nearest_nodes(G, X=-122.245846, Y=37.828903)
dest = ox.distance.nearest_nodes(G, X=-122.215006, Y=37.812303)

In [ ]:
# find the shortest path between nodes, minimizing travel time, then plot it
route = ox.shortest_path(G, orig, dest, weight="travel_time")
fig, ax = ox.plot_graph_route(G, route, node_size=0)

In [ ]:
# how long is our route in meters?
edge_lengths = ox.utils_graph.get_route_edge_attributes(G, route, "length")
round(sum(edge_lengths))

In [ ]:
# how far is it between these two nodes as the crow flies?
# use OSMnx's vectorized great-circle distance (haversine) function
orig_x = G.nodes[orig]["x"]
orig_y = G.nodes[orig]["y"]
dest_x = G.nodes[dest]["x"]
dest_y = G.nodes[dest]["y"]
round(ox.distance.great_circle_vec(orig_y, orig_x, dest_y, dest_x))

<a id="pois"></a>
## Points of Interest and Geospatial Entities
The `geometries` module of OSMnx allows downloading entities, such as grocery stores, transit stops, points of interest (POIs), or building footprints, and turn them into a `GeoDataFrame`.

To query, pass a tags dict where keys are OSM tags. The dict's values can be either:

- True to retrieve all OSM objects with this tag, regardless of its value
- a single value as a string to retrieve all OSM objects with this tag:value combination
- a list of string values to retrieve all OSM objects where this tag matches any of these values

Pass multiple dict key:value pairs to retrieve the union (not intersection) of these pairs.

See [docs](https://wiki.openstreetmap.org/wiki/Map_features).

In [ ]:
# get all building footprints in some neighborhood
# `True` means retrieve any object with this tag, regardless of value
place = "Aeroporto Galileo Galilei, Pisa, Italy"
#place = 'Bunker Hill, Los Angeles, California'
tags = {"building": True}
gdf = ox.geometries_from_place(place, tags)
gdf.shape

In [ ]:
fig, ax = ox.plot_footprints(gdf, figsize=(3, 3))

In [ ]:
# get all the parks in some neighborhood
# constrain acceptable `leisure` tag values to `park`
place = "Pisa, Italy"
tags = {"leisure": "park"}
gdf = ox.geometries_from_place(place, tags)
gdf.shape

In [ ]:
fig, ax = ox.plot_footprints(gdf, figsize=(9, 9))

In [ ]:
# get everything tagged amenity,
# and everything tagged landuse = retail or commercial,
# and everything tagged highway = bus_stop
tags = {"amenity": True, "landuse": ["retail", "commercial"], "highway": "bus_stop"}
gdf = ox.geometries_from_place("Piedmont, California, USA", tags)
gdf.shape

In [ ]:
fig, ax = ox.plot_footprints(gdf, figsize=(9, 9))

### Example: Let's take all restaurants in Lucca

In [ ]:
# get all the parks in some neighborhood
# constrain acceptable `leisure` tag values to `park`
tags = {"building": 'supermarket'}
gdf = ox.geometries_from_place('Lucca, Italy', tags)
gdf.shape

In [ ]:
gdf.head()

In [ ]:
gdf.plot()

In [ ]:
fig, ax = ox.plot_footprints(gdf, figsize=(9, 9))